In [2]:
api_key = 'JUQBES238PMPJKI2EJWAJJ2V8973EMCEZH'


import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import math
import time
import requests

from scipy.stats.mstats import winsorize

pd.set_option('display.float_format', '{:.2f}'.format)


def fetch_transaction_details(tx_id,
                              api_key,
                              etherscan_url = 'https://api.etherscan.io/api',
                              rate_limit=5):
    params = {
        'module': 'proxy',
        'action': 'eth_getTransactionByHash',
        'txhash': tx_id,
        'apikey': api_key
    }
        
    response = requests.get(etherscan_url, params=params)
    data = response.json()

    try:
        if 'input' in data['result']:
            del data['result']['input']
        return transaction_to_dataframe(data['result'])
    except:
        pass

def hex_to_int(hex_str):
    try:
        return int(hex_str, 16) if hex_str.startswith('0x') else hex_str
    except ValueError:
        return hex_str

# Function to convert transaction data to DataFrame
def transaction_to_dataframe(transaction_data):
    # Convert hex values to integers where applicable, but keep large numbers as strings
    for key, value in transaction_data.items():
        if isinstance(value, str) and value.startswith('0x'):
            if key in ['blockNumber', 'gas', 'nonce', 'transactionIndex', 'chainId']:
                transaction_data[key] = hex_to_int(value)
            else:
                # Keep the value as string if it's too large to handle as an integer
                try:
                    transaction_data[key] = hex_to_int(value)
                except OverflowError:
                    pass

    # Create a DataFrame from the dictionary
    df = pd.DataFrame([transaction_data])
    
    return df

In [ ]:
# Import Existing Transaction Number, this query pulls Supplemental Info when KNOWNING a Transction ID

In [14]:

df = pd.read_csv('Arbitrage_Files//arbitrage_20230611_20240613_WETH_USDC_57.csv')


tran1 = df[df['p1.transaction_id'].notnull()][['p1.transaction_id']].rename(columns={'p1.transaction_id':"TRANSACTION_ID"})
tran2 = df[df['p0.transaction_id'].notnull()][['p0.transaction_id']].rename(columns={'p0.transaction_id':"TRANSACTION_ID"})

final_tran = pd.concat([tran1,tran2]).drop_duplicates('TRANSACTION_ID')

# Test Single
#fetch_transaction_details(final_tran.iloc[0][0],api_key)

rate_limit = 5
start_time = time.time()
final_df = pd.DataFrame()

for count,transaction in enumerate(final_tran['TRANSACTION_ID']):
    if (count + 1) % rate_limit == 0:
        elapsed_time = time.time() - start_time
        if elapsed_time < 1:
            time.sleep(1 - elapsed_time)
        start_time = time.time()
    if count % 2500==0:
        print(count)
    final_df = pd.concat([final_df,fetch_transaction_details(transaction,api_key)])
    pass

0


In [52]:
for i in final_df.columns:
    final_df[i] = final_df[i].apply(lambda x:str(x))

final_df.to_csv('pool_supplemental_transactions_52.csv',index=False)